# Importando as bibliotecas necessárias

In [ ]:
const faker = require('faker');
const mongoose = require('mongoose');
const _ = require('lodash');

# Conectando a base de dados usando mongoose

In [ ]:
mongoose.connect('mongodb://localhost/effective_store', { useNewUrlParser: true }).then(() => console.log('Connected'));
console.log('Connecting...')

# Definindo o schema de Sellers

In [ ]:
const sellerSchema = new mongoose.Schema({
  name: String,
  country: String,
  productCatalog: [{
    name: String,
    price: Number
  }]
});

# Definindo o model de Seller

In [ ]:
const Seller = mongoose.model('Seller', sellerSchema);

# Gerando dados de sellers

In [ ]:
const SELLERS_LENGTH = 500;
const PRODUCTS_LENGTH = 100;

const availableProducts = _.range(PRODUCTS_LENGTH).map(() => ({
  name: faker.commerce.product(),
  price: parseFloat(faker.commerce.price())
}));

const sellers = _.range(SELLERS_LENGTH).map(() => {
  const productCatalogSize = _.random(1, PRODUCTS_LENGTH)
  const productCatalog = []
  _.range(productCatalogSize).forEach(() => {
    const productSample = _.sample(availableProducts)
    if (!productCatalog.includes(productSample)) {
      productCatalog.push(productSample)
    }
  });
  return new Seller({ 
    name: faker.company.companyName(),
    country: faker.address.country(),
    productCatalog
  });
});

# Populando base de dados

In [ ]:
Seller.deleteMany({}, () => console.log('cleared'));
Seller.insertMany(sellers, () => console.log('Inserted'));
console.log('Processing...')

# Testando um único resultado

In [ ]:
Seller.findOne({}, { _id: 0, productCatalog: { _id: 0 } }, (err, result) => console.log(result))
console.log('Processing...')

# Query 1 - Lojistas (Sellers) com maior número de produtos disponíveis na plataforma

In [ ]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      productCatalogSize: { $cond: { if: { $isArray: "$productCatalog" }, then: { $size: "$productCatalog" }, else: "NA"} }
    }
  },
  {
    $sort : { productCatalogSize : -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.log(result))
console.log('Processing...')

# Query 2 - Produtos mais comuns entre os lojistas associados

In [ ]:
Seller.aggregate([
  {
    $unwind: '$productCatalog'
  },
  {
    $group: {
      _id: '$productCatalog.name',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  }
], (errors, result) => console.log(result))
console.log('Processing...')

# Query 3 - Países com mais logistas

In [ ]:
Seller.aggregate([
  {
    $group: {
      _id: '$country',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.log(result))
console.log('Processing...')